<a href="https://colab.research.google.com/github/ixomaxip/nasoc/blob/master/src/BERTweet_k_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Aug 11 15:00:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture

!pip install nltk emoji
!pip install tweet-preprocessor

!pip install transformers
!pip install datasets

!wget https://hasocfire.github.io/hasoc/2021/files/en_Hasoc2021_train.zip
!unzip -P hasoc2021_en en_Hasoc2021_train.zip
!ls


In [ ]:
#@title
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#@title
# from google.colab import files
# uploaded = files.upload()

# !rm -rf /root/.ssh
# !mkdir /root/.ssh
# # !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz && chmod 700 /root/.ssh
# !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz
# !ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

# !git config --global user.email 'ixomaxip@gmail.com'
# !git config --global user.user 'ixomaxip'
# !ssh -T git@github.com

# !git clone git@github.com:ixomaxip/nasoc.git

# Custom BERTweet

In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import datetime
import re

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

#constants and parameters
seed = 42

path_gdrive_saved = '/content/drive/My Drive/nasoc/bert_tuned.weights'

batch_size = 8

DOWNLOAD_DATASET = False

# dataset

raw_df = pd.read_csv('en_Hasoc2021_train.csv')

raw_df['inputs'] = pd.Categorical(raw_df['task_1'])
raw_df['inputs'] = raw_df.inputs.cat.codes

raw_df['task_2_cat'] = pd.Categorical(raw_df['task_2'])
raw_df['task_2_cat'] = raw_df.task_2_cat.cat.codes

train_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=seed)

sr_train = train_df.groupby('task_1').count()['text']
sr_test  = test_df.groupby('task_1').count()['text']

tr_pr = sr_train['NOT']/sr_train['HOF']
ts_pr = sr_test['NOT']/sr_test['HOF']

#copied from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

# copied from https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    #tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(tweet.split())
    tweet = remove_emojis(tweet)
    #tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
    #     if w.lower() in words or not w.isalpha())
    return tweet

raw_df_tmp = raw_df.copy(True)
train_df, test_df = train_test_split(raw_df_tmp, test_size=0.2, random_state=seed)
#text = train_df.pop('text')#.map(lambda line: cleaner(line))
#label = train_df.pop('inputs')

#text_val = test_df.pop('text')#.map(lambda line: cleaner(line))
#label_val = test_df.pop('inputs')

#train_ds = tf.data.Dataset.from_tensor_slices((text.values, label.values))
#val_ds = tf.data.Dataset.from_tensor_slices((text_val.values, label_val.values))

#train_ds = train_ds.batch(batch_size=batch_size, drop_remainder=True)
#val_ds = val_ds.batch(batch_size=batch_size, drop_remainder=True)

In [1]:
import time, datetime
import random
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F, Linear
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, AutoModel, AdamW
from transformers import get_linear_schedule_with_warmup

## Dataloader

In [3]:
class HasocDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors='pt')
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#MODEL_NAME = "cardiffnlp/twitter-roberta-base-hate"
MODEL_NAME = "vinai/bertweet-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = HasocDataset(train_df['text'].to_list(), train_df['inputs'].to_list(), tokenizer)
test_dataset = HasocDataset(test_df['text'].to_list(), test_df['inputs'].to_list(), tokenizer)

raw_dataset = HasocDataset(raw_df['text'].to_list(), raw_df['inputs'].to_list(), tokenizer)

BATCH_SIZE = 16


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# it = iter(test_dataloader)
# first = next(it)

# print(first)

## Customization class & auxiliary functions

In [4]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# DEVICE = torch.device('cpu')

# Customization
class CustomBERTweet(nn.Module):
    def __init__(self):
        super(CustomBERTweet, self).__init__()
        self.bertweet = AutoModel.from_pretrained(MODEL_NAME)
        #self.cnn = nn.Conv1d(1, 768, 3)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(768, 2)
        # self.dense2 = nn.Linear(256, 2)
    
    def forward(self, feed_dict):

        bertweet_output = self.bertweet(**feed_dict)
        drop_output = self.dropout(bertweet_output.pooler_output)
        linear1_output = self.dense1(drop_output)
        # linear2_output = self.dense2(linear1_output)

        return linear1_output

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def forward(batch, model):
    feed_dict = {
        'input_ids': batch['input_ids'].to(DEVICE),
        'token_type_ids': batch['token_type_ids'].to(DEVICE),
        'attention_mask': batch['attention_mask'].to(DEVICE),
    }
    # feed_dict = {
    #     'input_ids': batch['input_ids'].to(DEVICE),
    #     'token_type_ids': batch['token_type_ids'].to(DEVICE),
    #     'attention_mask': batch['attention_mask'].to(DEVICE),
    # }    
    outputs = model(feed_dict)
    outputs = F.log_softmax(outputs, dim=1)

    return outputs

def validate(model, dataloader, as_dict=False):
    predictions, targets = [], []
    for batch in dataloader:
        target = batch['labels']
        with torch.no_grad():
            logits = forward(batch, model)
        logits = logits.detach().cpu().numpy()
        label = target.to('cpu').numpy()        
        predictions.append(logits)
        targets.append(label)
    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    flat_targets = [item for sublist in targets for item in sublist]
    return classification_report(flat_targets, flat_predictions, output_dict=as_dict)

def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
# del model

## Create model

In [5]:
true_train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = BATCH_SIZE
)
true_test_dataloader = DataLoader(
            test_dataset,
            sampler = RandomSampler(test_dataset),
            batch_size = BATCH_SIZE
)
len(true_train_dataloader)

#model = CustomBERTweet()

193

## Check model

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print(f'The BERT model has {len(params):} different named parameters.\n')
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')
print('\n==== Output Layer ====\n')
for p in params[-12:]:
    print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')

In [ ]:
model.to(torch.device('cuda'))
print(validate(model, test_dataloader))

## Train

In [6]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GroupKFold

k_folds = 2
EPOCHS = 1

# For fold results
results = {}

kfold = KFold(n_splits=k_folds, shuffle=True)


In [10]:
# # model = CustomBERTweet()
# # params = list(model.named_parameters())
# # model.to(torch.device('cuda'))
# # print(validate(model, test_dataloader))

for fold, (train_ids_idx, test_ids_idx) in enumerate(kfold.split(train_dataset)):    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
#     # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids_idx)
    
    # Define data loaders for training and testing data in this fold
    k_fold_train_dataloader = DataLoader(
            train_dataset,
            sampler = train_subsampler,
            batch_size = BATCH_SIZE
    )
    k_fold_test_dataloader = DataLoader(
            train_dataset,
            sampler = test_subsampler,
            batch_size = BATCH_SIZE
    )

    model_kf = CustomBERTweet()

    total_steps = len(k_fold_train_dataloader) * EPOCHS
    optimizer = AdamW(model_kf.parameters(), lr = 2e-5, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 20,
                                                num_training_steps = total_steps)
    loss_func = nn.CrossEntropyLoss()
    model_kf.to(DEVICE)

    #print(validate(model_kf, test_dataloader))

    start = time.time()
    for epoch in range(0, EPOCHS):
        total_train_loss = 0
        model_kf.train()
        print(f'======== Epoch {epoch+1} / {EPOCHS} ========')
        for step, batch in enumerate(k_fold_train_dataloader):
            #print(f"step: {step}")
            if step % 50 == 0 and not step == 0:
                elapsed = format_time(time.time() - start)
                print(f'\tBatch {step:>5,}  of  {len(k_fold_train_dataloader):>5,}.    Elapsed: {elapsed}.')
                # print(f'\tloss: {loss.item()}')
            optimizer.zero_grad()
            outputs = forward(batch, model_kf)
            target = batch['labels'].to(DEVICE)
            model_kf.zero_grad()
            loss = loss_func(outputs, target)
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_kf.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        
        avg_train_loss = total_train_loss / len(k_fold_test_dataloader)
        training_time = format_time(time.time() - start)
        print(f'\tAverage training loss: {avg_train_loss:.2f}')
        print(f'\tTraining epoch took: {training_time}')
        model_kf.eval()
        report = validate(model_kf, k_fold_test_dataloader)
        print(f'======== Validation report {epoch+1} / {EPOCHS} ========')
        print(report)

    avg_train_loss = total_train_loss / len(true_test_dataloader)
    training_time = format_time(time.time() - start)
    model_kf.eval()
    report = validate(model_kf, true_test_dataloader)
    print(f'======== fold {fold} validation ========')
    print(report)


FOLD 0
--------------------------------


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


======== Epoch 1 / 4 ========


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


	Batch    50  of    154.    Elapsed: 0:00:07.
	Batch   100  of    154.    Elapsed: 0:00:13.
	Batch   150  of    154.    Elapsed: 0:00:20.
	Average training loss: 2.13
	Training epoch took: 0:00:20
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.75      0.95      0.84       398
           1       0.82      0.43      0.56       217

    accuracy                           0.77       615
   macro avg       0.79      0.69      0.70       615
weighted avg       0.78      0.77      0.74       615

======== Epoch 2 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:28.
	Batch   100  of    154.    Elapsed: 0:00:34.
	Batch   150  of    154.    Elapsed: 0:00:41.
	Average training loss: 1.58
	Training epoch took: 0:00:41
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       398
           1       0.81      0.55      0.66       217

 

...

======== Epoch 1 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:07.
	Batch   100  of    154.    Elapsed: 0:00:13.
	Batch   150  of    154.    Elapsed: 0:00:20.
	Average training loss: 2.14
	Training epoch took: 0:00:20
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.85      0.82      0.84       389
           1       0.71      0.75      0.73       226

    accuracy                           0.80       615
   macro avg       0.78      0.79      0.78       615
weighted avg       0.80      0.80      0.80       615

======== Epoch 2 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:28.
	Batch   100  of    154.    Elapsed: 0:00:34.
	Batch   150  of    154.    Elapsed: 0:00:41.
	Average training loss: 1.52
	Training epoch took: 0:00:41
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       389
           1       0.84   

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


======== Epoch 1 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:06.
	Batch   100  of    154.    Elapsed: 0:00:13.
	Batch   150  of    154.    Elapsed: 0:00:19.
	Average training loss: 2.17
	Training epoch took: 0:00:20
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.74      0.98      0.84       390
           1       0.91      0.41      0.57       225

    accuracy                           0.77       615
   macro avg       0.83      0.70      0.71       615
weighted avg       0.80      0.77      0.74       615

======== Epoch 2 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:28.
	Batch   100  of    154.    Elapsed: 0:00:34.
	Batch   150  of    154.    Elapsed: 0:00:41.
	Average training loss: 1.62
	Training epoch took: 0:00:41
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       390
           1       0.79   

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


======== Epoch 1 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:06.
	Batch   100  of    154.    Elapsed: 0:00:13.
	Batch   150  of    154.    Elapsed: 0:00:19.
	Average training loss: 2.23
	Training epoch took: 0:00:20
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       407
           1       0.79      0.57      0.66       207

    accuracy                           0.80       614
   macro avg       0.80      0.74      0.76       614
weighted avg       0.80      0.80      0.79       614

======== Epoch 2 / 4 ========
	Batch    50  of    154.    Elapsed: 0:00:28.
	Batch   100  of    154.    Elapsed: 0:00:34.
	Batch   150  of    154.    Elapsed: 0:00:41.
	Average training loss: 1.65
	Training epoch took: 0:00:41
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.83      0.91      0.87       407
           1       0.78   

In [ ]:
model.eval()
report = validate(model, test_dataloader)
print(report)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


              precision    recall  f1-score   support

           0       0.86      0.91      0.88       529
           1       0.77      0.68      0.72       240

    accuracy                           0.84       769
   macro avg       0.82      0.80      0.80       769
weighted avg       0.83      0.84      0.83       769



#Garbage

In [ ]:
from transformers import Trainer, TrainingArguments
OUTPUT_DIR = os.path.join(ROOT_DIR, 'labse_ft_results')

training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=os.path.join(OUTPUT_DIR, 'logs'),            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=custom_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)
trainer.train()

***** Running training *****
  Num examples = 3074
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 579
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


TypeError: ignored

In [ ]:
max_len = 64
row = train_df.iloc[0]
texts = row.text
encoded_input = tokenizer([texts, texts], padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(device=DEVICE)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)

In [ ]:
model_output.pooler_output.shape

torch.Size([2, 768])

In [ ]:
train_embedings = text_encoder(train_texts, model, tokenizer, 64)

In [ ]:
sentences = ["Hello World"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(DEVICE)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
# print(embeddings[0])

In [ ]:
model_output.pooler_output.shape

torch.Size([1, 768])

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7f24480df4d0>

In [ ]:
len(train_labels)

3074

In [ ]:


train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)